## 從104人力銀行分析 該職缺相關技能

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import datetime
import re
import jieba
from selenium import webdriver
from selenium.webdriver.support.ui import Select


### 函式定義

In [3]:
def getSoup(url):
    resp=requests.get(url)
    resp.encoding='utf-8'
    if resp.status_code==200:
        soup=BeautifulSoup(resp.text,'lxml')
        return soup
    return None

In [9]:
def getWebdriver(url,options=None,wait=10):
    if options==None:
        options=webdriver.ChromeOptions()
        options.add_argument('--headless')
    try:
        driver=webdriver.Chrome(r'C:\Users\LIANG\Desktop\profilio_sourcecode\2_104analysis\chromedriver',options=options)
        driver.implicitly_wait(wait)
        driver.get(url)
    except:
        print('not found')
    return driver

### 找最大頁數

In [12]:
url='https://www.104.com.tw/jobs/main/'
driver=getWebdriver(url)
driver.find_element_by_xpath('/html/body/article[1]/div/div/div[4]/div/input').send_keys('scala\n')

time.sleep(1)  # 不能省略

soup=BeautifulSoup(driver.page_source,'lxml')
max_page=int(re.findall('\s(\d)+\s\W',soup.find('select',class_="page-select js-paging-select gtm-paging-top").find_all('option')[-1].text)[0])
max_page

6

### 抓取第一頁資料

In [13]:
url_api='https://www.104.com.tw/jobs/search/'
params={
    'ro': '0',
    'keyword': 'scala',
    'jobcatExpansionType': '0',
    'order': '14',
    'asc': '0',
    'page': '1',
    'mode': 's',
    'jobsource': '2018indexpoc'
    }


page=1

resp=requests.get(url_api,params)
soup=BeautifulSoup(resp.text,'lxml')
datas=soup.find(id="js-job-content").find_all('article')

# 一頁20筆資料
len(soup.find(id="js-job-content").find_all('article'))

20

### 抓取子頁面資料

In [15]:
url_subpage='https://www.104.com.tw/job/6qwxd?jobsource=2018indexpoc'

driver=getWebdriver(url_subpage)
time.sleep(1)
soup=BeautifulSoup(driver.page_source,'lxml')

# 抓取 '工作內容' 及 '其他條件'
content=soup.find('p',class_="mb-5 r3 job-description__content text-break").text.strip()
condition=soup.find('p',class_="m-0 r3").text.strip()

### 子頁面文字探勘

In [19]:

keyword_list=[]
keyword=re.sub(r'[0-9.,\r\n\t+]','',content+condition)
keyword = jieba.cut(keyword, cut_all=True)
for k in keyword:
    if k!=' ' and k!='':
        keyword_list.append(k)

### 抓出每頁資料

In [73]:
#################
url='https://www.104.com.tw/jobs/main/'
chrome=webdriver.Chrome(r'C:\Users\LIANG\Desktop\profilio_sourcecode\2_104analysis\chromedriver')
chrome.implicitly_wait(10)
chrome.get(url)
chrome.find_element_by_xpath('/html/body/article[1]/div/div/div[4]/div/input').send_keys('scala\n')

time.sleep(1)  # 不能省略

soup=BeautifulSoup(chrome.page_source,'lxml')
max_page=int(re.findall('\s(\d)+\s\W',soup.find('select',class_="page-select js-paging-select gtm-paging-top").find_all('option')[-1].text)[0])
chrome.quit()



final_data=[]

# 抓取每頁資料
for page in range(1,max_page+1):
    params={
    'ro': '0',
    'keyword': 'scala',
    'jobcatExpansionType': '0',
    'order': '14',
    'asc': '0',
    'page': page,
    'mode': 's',
    'jobsource': '2018indexpoc'
    }
    resp=requests.get(url_api,params)
    soup=BeautifulSoup(resp.text,'lxml')
    datas=soup.find(id="js-job-content").find_all('article')
    
    # 抓取每頁職缺資料
    for data in datas:
        temp=[]
        title=data.find('h2').find('a').text.strip()
        link='https:'+data.find('h2').find('a')['href']
        company=data.find('ul',class_="b-list-inline b-clearfix").find_all('li')[1].text.strip()
        industrial=data.find('ul',class_="b-list-inline b-clearfix").find_all('li')[2].text.strip()
        area=data.find('ul',class_="b-list-inline b-clearfix job-list-intro b-content").find_all('li')[0].text.strip()
        experience=data.find('ul',class_="b-list-inline b-clearfix job-list-intro b-content").find_all('li')[1].text.strip()
        education=data.find('ul',class_="b-list-inline b-clearfix job-list-intro b-content").find_all('li')[2].text.strip()
        pay=data.find('div',class_="job-list-tag b-content").find_all('span')[0].text.strip()

        # 尋找該職位Keyword
        # 子頁面有javascript，所以用seleniun
        chrome=getWebdriver(link)
        time.sleep(0.1)
        soup=BeautifulSoup(chrome.page_source,'lxml')
        try:
            content=soup.find('p',class_="mb-5 r3 job-description__content text-break").text.strip()
            condition=soup.find('p',class_="m-0 r3").text.strip()
        except Exception  as e:
            print(e)
        chrome.quit()
        # jieba( string , mode )
        keyword_list=[]
        if content and condition !=[]:
            keyword=re.sub(r'[0-9.,\r\n\t+]','',content+condition)
            keyword = jieba.cut(keyword, cut_all=True)
            for k in keyword:
                if k!=' 'and k!='':
                    keyword_list.append(k)

        temp.extend([title,link,company,industrial,area,experience,education,pay])
        temp.append(keyword_list)
        final_data.append(temp)
    print(f'第{page}頁完成')


第1頁完成
第2頁完成
第3頁完成
第4頁完成
第5頁完成
第6頁完成


[['Senior Java/Scala Data Backend Engineer (大數據)',
  'https://www.104.com.tw/job/6qwxd?jobsource=2018indexpoc',
  '伊諾科技有限公司',
  '電腦系統整合服務業',
  '台北市內湖區',
  '4年以上',
  '專科',
  '待遇面議',
  ['Design',
   'develop',
   'and',
   'maintain',
   'Big',
   'Data',
   'related',
   'projects',
   'including',
   'EDW',
   'system',
   'and',
   'ETL',
   'pipeline',
   'for',
   'data',
   'access',
   'collection',
   'processing',
   'and',
   'storageManage',
   'deployment',
   'of',
   'the',
   'Big',
   'Data',
   'Infrastructure',
   'and',
   'services',
   'on',
   'AWSDedicate',
   'to',
   'Big',
   'Data',
   'and',
   'Machine',
   'Learning',
   'Platform',
   'using',
   'Apache',
   'Spark',
   'and',
   'related',
   'technologiesInvestigate',
   'survey',
   'and',
   'evaluate',
   'various',
   'state',
   '-',
   'of',
   '-',
   'the',
   '-',
   'art',
   'technologiesIntroduce',
   'new',
   'technologies',
   'and',
   'tackle',
   'the',
   'derived',
   'problemsImpleme

In [74]:
len(page_data)

104

In [80]:
df=pd.DataFrame(page_data,columns=['Title','Link','company','Industrial','Area','Experience','Education','Pay','Keyword'])
df.to_csv('scala.csv',encoding='utf-8-sig')


In [18]:
# jieba( string , mode )
keyword_list=[]
keyword=re.sub(r'[0-9.,\r\n\t+]','',content+condition)
keyword = jieba.cut(keyword, cut_all=True)
for k in keyword:
    if k!=' ' and k!='':
        keyword_list.append(k)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LIANG\AppData\Local\Temp\jieba.cache
Loading model cost 0.520 seconds.
Prefix dict has been built successfully.
